In [3]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings(action='ignore')
import nltk
#nltk.download('all') # 처음 실행 시 주석을 제거하고 실행해주세요!

In [4]:
data = pd.read_csv('train.csv') # train 데이터 불러오기
data.head() # 데이터 확인하기

,id,document,label
0,1,영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐,0
1,2,히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯...,1
2,3,괜찮은 음악영화가 또 나왔군요!!! 따뜻한 겨울이 될 것 같아요~,1
3,4,아무래도 20년도지난작품이라 지금보기는너무유치하다,0
4,5,지금까지의 영화들이 그랬듯. 이 영화역시 일본에 대한 미화는 여전하다.,0


In [5]:
data.isna().sum() # 결측치 확인

id          0
document    0
label       0
dtype: int64

In [6]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(data)
train.reset_index(inplace=True) # 전처리 과정에서 데이터가 뒤섞이지 않도록 인덱스를 초기화해주었습니다.
val.reset_index(inplace=True)

train # train 셋 확인

,index,id,document,label
0,2224,2225,더빙으로 보느니 차라리 소리를 끄고 보는게 좋을뻔했습니다.,0
1,3479,3480,"내 유년시절엔 그래도 호소자, 칠소복만큼 재밌는 영화가 없었다",1
2,1206,1207,오백원준다니 일단 적어근데 보기전에 적으라는건 뭐냐ㅋㅋ보고나서적으라고 뜨던가,1
3,1330,1331,감동도 없고.기독교음악.기독교자막 이건아니잖아요~!,0
4,2264,2265,보고싶다. 줄거리는 일단 재밌는 듯...???,1
...,...,...,...,...
3745,4990,4991,내머리속에미달이....흠 미달이 벗얻?ㄷ 애로틱하지않음,0
3746,1141,1142,뭐냐 이영화 왜 0점은 줄수없는거냐 글고 여고2 너무따라해서 나열받음,0
3747,3537,3538,ㅋㅋ첨으로 네이버평점에 동의한다 ㅅㅂ 그냥 초등학생들이 보면되겠다,0
3748,1579,1580,하지원의 화려한 모습 정말 아름답다.,1


In [7]:
train['preprocessed'] = train['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글과 공백을 제외하고 모두 제거
train[:5] # 데이터 확인

,index,id,document,label,preprocessed
0,2224,2225,더빙으로 보느니 차라리 소리를 끄고 보는게 좋을뻔했습니다.,0,더빙으로 보느니 차라리 소리를 끄고 보는게 좋을뻔했습니다
1,3479,3480,"내 유년시절엔 그래도 호소자, 칠소복만큼 재밌는 영화가 없었다",1,내 유년시절엔 그래도 호소자 칠소복만큼 재밌는 영화가 없었다
2,1206,1207,오백원준다니 일단 적어근데 보기전에 적으라는건 뭐냐ㅋㅋ보고나서적으라고 뜨던가,1,오백원준다니 일단 적어근데 보기전에 적으라는건 뭐냐ㅋㅋ보고나서적으라고 뜨던가
3,1330,1331,감동도 없고.기독교음악.기독교자막 이건아니잖아요~!,0,감동도 없고기독교음악기독교자막 이건아니잖아요
4,2264,2265,보고싶다. 줄거리는 일단 재밌는 듯...???,1,보고싶다 줄거리는 일단 재밌는 듯


In [8]:
train['preprocessed'] = train['preprocessed'].str.replace(" +", " ") # 다중 공백 제거
train[:5] # 데이터 확인

,index,id,document,label,preprocessed
0,2224,2225,더빙으로 보느니 차라리 소리를 끄고 보는게 좋을뻔했습니다.,0,더빙으로 보느니 차라리 소리를 끄고 보는게 좋을뻔했습니다
1,3479,3480,"내 유년시절엔 그래도 호소자, 칠소복만큼 재밌는 영화가 없었다",1,내 유년시절엔 그래도 호소자 칠소복만큼 재밌는 영화가 없었다
2,1206,1207,오백원준다니 일단 적어근데 보기전에 적으라는건 뭐냐ㅋㅋ보고나서적으라고 뜨던가,1,오백원준다니 일단 적어근데 보기전에 적으라는건 뭐냐ㅋㅋ보고나서적으라고 뜨던가
3,1330,1331,감동도 없고.기독교음악.기독교자막 이건아니잖아요~!,0,감동도 없고기독교음악기독교자막 이건아니잖아요
4,2264,2265,보고싶다. 줄거리는 일단 재밌는 듯...???,1,보고싶다 줄거리는 일단 재밌는 듯


In [2]:
from konlpy.tag import Okt 

han_sentence = "오늘도 열심히 코딩을 해볼까요? 같이 힘내서 자연어 처리 고수가 됩시다! ㅎㅎ"
okt = Okt() # 인스턴스 할당
print("한국어 형태소 분석 결과(어간 추출X) ==>", okt.morphs(han_sentence, stem = False)) # 형태소 단위로 분리
print("한국어 형태소 분석 결과(어간 추출O) ==>", okt.morphs(han_sentence, stem = True)) # 형태소 단위로 분리 후 어간 추출

한국어 형태소 분석 결과(어간 추출X) ==> ['오늘', '도', '열심히', '코딩', '을', '해볼까', '요', '?', '같이', '힘내서', '자연어', '처리', '고수', '가', '됩시다', '!', 'ㅎㅎ']
한국어 형태소 분석 결과(어간 추출O) ==> ['오늘', '도', '열심히', '코딩', '을', '해보다', '요', '?', '같이', '힘내다', '자연어', '처리', '고수', '가', '되다', '!', 'ㅎㅎ']


In [9]:
tokenized = [] # 데이터프레임의 한 컬럼으로 추가할 리스트
for sentence in train['preprocessed']: # 전처리된 리뷰들을 하나씩 꺼내옵니다
    tokens = okt.morphs(sentence, stem = True) # 형태소 분석 (stem = True로 설정해 어간 추출을 해주었습니다)
    tokenize = " ".join(tokens) # tokens라는 리스트 안의 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join시켜줍니다.
    tokenized.append(tokenize) # 형태소 단위로 띄어쓰기된 문자열을 최종 리스트에 추가해줍니다
train["tokenized_stem"] = pd.DataFrame(tokenized) # 리스트를 데이터프레임으로 변환해 tokenized_stem라는 컬럼명으로 추가해줍니다.

train.head() # 데이터 확인

,index,id,document,label,preprocessed,tokenized_stem
0,2224,2225,더빙으로 보느니 차라리 소리를 끄고 보는게 좋을뻔했습니다.,0,더빙으로 보느니 차라리 소리를 끄고 보는게 좋을뻔했습니다,더빙 으로 보다 차라리 소리 를 끄다 보다 좋다 뻔하다
1,3479,3480,"내 유년시절엔 그래도 호소자, 칠소복만큼 재밌는 영화가 없었다",1,내 유년시절엔 그래도 호소자 칠소복만큼 재밌는 영화가 없었다,내 유년 시절 엔 그래도 호소 자 칠 소복 만큼 재밌다 영화 가 없다
2,1206,1207,오백원준다니 일단 적어근데 보기전에 적으라는건 뭐냐ㅋㅋ보고나서적으라고 뜨던가,1,오백원준다니 일단 적어근데 보기전에 적으라는건 뭐냐ㅋㅋ보고나서적으라고 뜨던가,오백원 주다 일단 적다 근데 보다 적다 건 뭐 냐 ㅋㅋ 보다 적다 뜨다
3,1330,1331,감동도 없고.기독교음악.기독교자막 이건아니잖아요~!,0,감동도 없고기독교음악기독교자막 이건아니잖아요,감동 도 없다 기독교 음악 기독교 자막 이건 아니다
4,2264,2265,보고싶다. 줄거리는 일단 재밌는 듯...???,1,보고싶다 줄거리는 일단 재밌는 듯,보다 줄거리 는 일단 재밌다 듯


In [10]:
# 품사 태깅
print(okt.pos("오늘도 열심히 재밌는 코딩을 해볼까? 같이 힘내서 자연어 처리 고수들이 됩시다! ㅎㅎ"))

[('오늘', 'Noun'), ('도', 'Josa'), ('열심히', 'Adverb'), ('재밌는', 'Adjective'), ('코딩', 'Noun'), ('을', 'Josa'), ('해볼까', 'Verb'), ('?', 'Punctuation'), ('같이', 'Adverb'), ('힘내서', 'Verb'), ('자연어', 'Noun'), ('처리', 'Noun'), ('고수', 'Noun'), ('들', 'Suffix'), ('이', 'Josa'), ('됩시다', 'Verb'), ('!', 'Punctuation'), ('ㅎㅎ', 'KoreanParticle')]


In [11]:
print(okt.pos("이것은 1점이 아니다 11점을 주고싶은 내 간절한 마음이다"))

[('이', 'Determiner'), ('것', 'Noun'), ('은', 'Josa'), ('1', 'Number'), ('점', 'Noun'), ('이', 'Josa'), ('아니다', 'Adjective'), ('11', 'Number'), ('점', 'Noun'), ('을', 'Josa'), ('주고싶은', 'Verb'), ('내', 'Noun'), ('간절한', 'Adjective'), ('마음', 'Noun'), ('이다', 'Josa')]


In [12]:
print(okt.nouns("오늘도 열심히 재밌는 코딩을 해볼까? 같이 힘내서 자연어 처리 고수들이 됩시다! ㅎㅎ"))

['오늘', '코딩', '자연어', '처리', '고수']


In [13]:
main_pos = [] # 데이터프레임의 새 컬럼이 될 리스트
for sentence in train['document']: # 리뷰들을 하나씩 가져옵니다
    pos = okt.pos(sentence) # 형태소 분석을 진행하고 해당 리스트를 pos라는 변수로 받습니다
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")] # 가져오고자 하는 품사에 해당하면 해당 형태소를 main_words 리스트에 추가합니다.
    main_words_str = " ".join(main_words) # main_words 리스트 안의 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join시켜줍니다.
    main_pos.append(main_words_str) # 선택한 형태소들로 이루어진 문자열을 최종 리스트에 추가해줍니다
train["main_pos"] = pd.DataFrame(main_pos) # 리스트를 데이터프레임으로 변환해 main_pos라는 컬럼명으로 추가해줍니다.

train.head() # 데이터 확인

,index,id,document,label,preprocessed,tokenized_stem,main_pos
0,2224,2225,더빙으로 보느니 차라리 소리를 끄고 보는게 좋을뻔했습니다.,0,더빙으로 보느니 차라리 소리를 끄고 보는게 좋을뻔했습니다,더빙 으로 보다 차라리 소리 를 끄다 보다 좋다 뻔하다,더빙 보느니 차라리 소리 끄고 보는게 좋을 뻔했습니다
1,3479,3480,"내 유년시절엔 그래도 호소자, 칠소복만큼 재밌는 영화가 없었다",1,내 유년시절엔 그래도 호소자 칠소복만큼 재밌는 영화가 없었다,내 유년 시절 엔 그래도 호소 자 칠 소복 만큼 재밌다 영화 가 없다,내 유년 시절 그래도 호소 소복 재밌는 영화 없었다
2,1206,1207,오백원준다니 일단 적어근데 보기전에 적으라는건 뭐냐ㅋㅋ보고나서적으라고 뜨던가,1,오백원준다니 일단 적어근데 보기전에 적으라는건 뭐냐ㅋㅋ보고나서적으라고 뜨던가,오백원 주다 일단 적다 근데 보다 적다 건 뭐 냐 ㅋㅋ 보다 적다 뜨다,오백원 준다니 일단 적어 근데 보기전에 적으라는 건 뭐 보고나서 적으라고 뜨던가
3,1330,1331,감동도 없고.기독교음악.기독교자막 이건아니잖아요~!,0,감동도 없고기독교음악기독교자막 이건아니잖아요,감동 도 없다 기독교 음악 기독교 자막 이건 아니다,감동 없고 기독교 음악 기독교 자막 이건 아니잖아요
4,2264,2265,보고싶다. 줄거리는 일단 재밌는 듯...???,1,보고싶다 줄거리는 일단 재밌는 듯,보다 줄거리 는 일단 재밌다 듯,보고싶다 줄거리 일단 재밌는 듯


In [14]:
X_train = train.main_pos #training 데이터에서 문서 추출
y_train = train.label #training 데이터에서 라벨 추출

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer() #countvectorizer 생성
vectorizer.fit(X_train) # countvectorizer 학습
X_train_vec = vectorizer.transform(X_train) # transform

In [16]:
from sklearn.linear_model import LogisticRegression #모델 불러오기
model = LogisticRegression() #객체에 모델 할당
model.fit(X_train_vec, y_train) #모델 학습

LogisticRegression()

In [17]:
# preprocessed
val['preprocessed'] = val['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글과 공백을 제외하고 모두 제거
val['preprocessed'] = val['preprocessed'].str.replace(" +", " ") # 다중 공백 제거

# tokenized_stem
tokenized = []
for sentence in val['preprocessed']:
    tokens = okt.morphs(sentence, stem = True) # stem = True로 설정해 어간 추출을 해주었습니다
    tokenize = " ".join(tokens)
    tokenized.append(tokenize)
val["tokenized_stem"] = pd.DataFrame(tokenized)

# main_pos
main_pos = []
for sentence in val['document']:
    pos = okt.pos(sentence)
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")]
    main_words_str = " ".join(main_words)
    main_pos.append(main_words_str)
val["main_pos"] = pd.DataFrame(main_pos)

val.head()

,index,id,document,label,preprocessed,tokenized_stem,main_pos
0,222,223,이게 뭔 액션이냐........ 개차반이네 ㅋㅋㅋㅋㅋㅋ,0,이게 뭔 액션이냐 개차반이네 ㅋㅋㅋㅋㅋㅋ,이 게 뭔 액션 이냐 개차반 이네 ㅋㅋㅋㅋㅋㅋ,이 액션 개차반
1,3488,3489,안타깝다 누군가에겐 사랑의 축복 누군가에겐 더 없을 저주 다이아몬드.,1,안타깝다 누군가에겐 사랑의 축복 누군가에겐 더 없을 저주 다이아몬드,안타깝다 누군가 에겐 사랑 의 축복 누군가 에겐 더 없다 저주 다이아몬드,안타깝다 누군가 사랑 축복 누군가 더 없을 저주 다이아몬드
2,4085,4086,1점도아까운 OO영화 이거 보느라 2시간날린 내가 불쌍하다,0,점도아까운 영화 이거 보느라 시간날린 내가 불쌍하다,점도 아깝다 영화 이 거 보다 시간 날리다 내 가 불쌍하다,점도 아까운 영화 거 보느라 내 불쌍하다
3,3545,3546,이 영화를 보고 머리에 남는 하나의 기억 '외계인의 턱에 달린.......,0,이 영화를 보고 머리에 남는 하나의 기억 외계인의 턱에 달린,이 영화 를 보고 머리 에 남다 하나 의 기억 외계인 의 턱 에 달리다,이 영화 보고 머리 남는 하나 기억 외계인 턱 달린
4,2131,2132,굶주린 총각이 재미있게 볼수 있을법한 영화,0,굶주린 총각이 재미있게 볼수 있을법한 영화,굶주리다 총각 이 재미있다 보다 있다 법 한 영화,굶주린 총각 재미있게 볼수 있을 법 영화


In [18]:
X_val = val.main_pos #validation 데이터에서 전처리된 문서 추출
y_val = val.label #validation 데이터에서 라벨 추출

X_val_vec = vectorizer.transform(X_val) # train셋으로 fit한 벡터라이저 이용해 transform

In [19]:
#run model
y_pred = model.predict(X_val_vec)
print(y_pred)
#0:negative 1:positive

[0 1 0 ... 0 1 0]


In [20]:
from sklearn import metrics
print('accuracy =', metrics.accuracy_score(y_val, y_pred)) # 정확도 확인

accuracy = 0.828


In [21]:
test = pd.read_csv('test.csv')
test

,id,document
0,1,시간 때우기 좋은 영화 지루함
1,2,훈훈한 정이 느껴지는 영화! 가족끼리 드라마 보듯이 보면 딱~!
2,3,Childhood fantasy
3,4,멋있는 영화입니다. 잊을 수 없는!
4,5,너무 감동적이네요 펑펑 울었습니다
...,...,...
4995,4996,연계도 제대로 안되고 뭔가 부족한느낌이들고 찝찝한영화
4996,4997,내생에 최악의 영화중 하나 졸려서 미칠뻔....
4997,4998,지금까지 나온 드라마 중 최고였습니다. 100점 만점!
4998,4999,영화값으로 엄청난 공연을 보았다 - 이게 웬 횡재 ..


In [22]:
test.isna().sum()

id          0
document    0
dtype: int64

In [23]:
# preprocessed
test['preprocessed'] = test['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글과 공백을 제외하고 모두 제거
test['preprocessed'] = test['preprocessed'].str.replace(" +", " ") # 다중 공백 제거

# tokenized_stem
tokenized = []
for sentence in test['preprocessed']:
    tokens = okt.morphs(sentence, stem = True) # stem = True로 설정해 어간 추출을 해주었습니다
    tokenize = " ".join(tokens)
    tokenized.append(tokenize)
test["tokenized_stem"] = pd.DataFrame(tokenized)

# main_pos
main_pos = []
for sentence in test['document']:
    pos = okt.pos(sentence)
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")]
    main_words_str = " ".join(main_words)
    main_pos.append(main_words_str)
test["main_pos"] = pd.DataFrame(main_pos)

test.head()

,id,document,preprocessed,tokenized_stem,main_pos
0,1,시간 때우기 좋은 영화 지루함,시간 때우기 좋은 영화 지루함,시간 때우다 좋다 영화 지루함,시간 때우기 좋은 영화 지루함
1,2,훈훈한 정이 느껴지는 영화! 가족끼리 드라마 보듯이 보면 딱~!,훈훈한 정이 느껴지는 영화 가족끼리 드라마 보듯이 보면 딱,훈훈하다 정이 느껴지다 영화 가족 끼리 드라마 보다 보다 딱,훈훈한 정이 느껴지는 영화 가족 끼리 드라마 보듯이 보면 딱
2,3,Childhood fantasy,,,
3,4,멋있는 영화입니다. 잊을 수 없는!,멋있는 영화입니다 잊을 수 없는,멋있다 영화 이다 잊다 수 없다,멋있는 영화 입니다 잊을 수 없는
4,5,너무 감동적이네요 펑펑 울었습니다,너무 감동적이네요 펑펑 울었습니다,너무 감동 적다 펑펑 울다,너무 감동 적이네요 펑펑 울었습니다


In [24]:
X_test = test.main_pos
X_test_vec = vectorizer.transform(X_test)
pred_test = model.predict(X_test_vec)
print(pred_test)

[0 1 0 ... 1 0 1]


In [25]:
# 제출용 sample 파일을 불러옵니다.
submission = pd.read_csv('sample_submission.csv')
submission.head()

,id,label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [26]:
# 위에서 구한 예측값을 그대로 넣어줍니다.
submission['label'] = pred_test

# 데이터가 잘 들어갔는지 확인합니다.
submission

,id,label
0,1,0
1,2,1
2,3,0
3,4,1
4,5,1
...,...,...
4995,4996,0
4996,4997,0
4997,4998,1
4998,4999,0


In [27]:
# submission을 csv 파일로 저장합니다.
# index=False란 추가적인 id를 부여할 필요가 없다는 뜻입니다. 
# 정확한 채점을 위해 꼭 index=False를 넣어주세요.
#submission.to_csv("submission_baseline2.csv", index=False)